In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 处理缺失数据
在许多数据分析⼯作中，缺失数据是经常发⽣的。`pandas`的⽬标之⼀就是尽量轻松地处理缺失数据。例如，`pandas`对象的所有描述性统计默认都不包括缺失数据。  
缺失数据在`pandas`中呈现的⽅式有些不完美，但对于⼤多数⽤户可以保证功能正常。对于数值数据，`pandas`使⽤浮点值`NaN`（Not a Number）表示缺失数据。我们称其为哨兵值，可以⽅便的检测出来：

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

在`pandas`中，我们采⽤了R语⾔中的惯⽤法，即将缺失值表示为`NA`，它表示不可⽤`not available`。在统计应⽤中，`NA`数据可能是不存在的数据或者虽然存在，但是没有观察到（例如，数据采集中发⽣了问题）。当进⾏数据清洗以进⾏分析时，最好直接对缺失数据进⾏分析，以判断数据采集的问题或缺失数据可能导致的偏差。  
`Python`内置的`None`值在对象数组中也可以作为`NA`：

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

`pandas`项⽬中还在不断优化内部细节以更好处理缺失数据，像⽤户`API`功能，例如`pandas.isnull`，去除了许多恼⼈的细节。  
![](./NA处理方法.jpg)

## 滤除缺失数据
过滤掉缺失数据的办法有很多种。你可以通过`pandas.isnull`或布尔索引的⼿⼯⽅法，但`dropna`可能会更实⽤⼀些。对于⼀个
`Series`，`dropna`返回⼀个仅含⾮空数据和索引值的`Series`：

In [5]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

这等价于：

In [6]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

⽽对于`DataFrame`对象，事情就有点复杂了。你可能希望丢弃全`NA`或含有`NA`的⾏或列。`dropna`默认丢弃任何含有缺失值的⾏：

In [7]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [8]:
cleaned

,0,1,2
0,1.0,6.5,3.0


传⼊how='all'将只丢弃全为NA的那些⾏：

In [9]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


⽤这种⽅式丢弃列，只需传⼊`axis=1`即可：

In [10]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [11]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


另⼀个滤除`DataFrame`⾏的问题涉及时间序列数据。假设你只想留下⼀部分观测数据，可以⽤`thresh`参数实现此⽬的：

In [12]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [13]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [14]:
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


## 填充缺失数据
你可能不想滤除缺失数据（有可能会丢弃跟它有关的其他数据），⽽是希望通过其他⽅式填补那些“空洞”。对于⼤多数情况⽽⾔，`fillna`⽅法是最主要的函数。通过⼀个常数调⽤`fillna`就会将缺失值替换为那个常数值：

In [15]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


若是通过⼀个字典调⽤`fillna`，就可以实现对不同的列填充不同的值：

In [16]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


`fillna`默认会返回新对象，但也可以对现有对象进⾏就地修改：

In [19]:
_ = df.fillna(0, inplace=True)  # 修改原数据
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


对`reindexing`有效的那些插值⽅法也可⽤于`fillna`：

In [20]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [21]:
df.fillna(method='ffill')

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,0.124121,-2.370232
5,-1.265934,0.124121,-2.370232


In [22]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


只要有些创新，你就可以利⽤`fillna`实现许多别的功能。⽐如说，你可以传⼊`Series`的平均值或中位数：

In [23]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

![](./fillna函数的参数.png)

## 移除重复数据
`DataFrame`中出现重复⾏有多种原因。下⾯就是⼀个例⼦：

In [30]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


`DataFrame`的`duplicated`⽅法返回⼀个布尔型`Series`，表示各⾏是否是重复⾏（前⾯出现过的⾏）：

In [31]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

还有⼀个与此相关的`drop_duplicates`⽅法，它会返回⼀个`DataFrame`，重复的数组会标为`False`：

In [32]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这两个⽅法默认会判断全部列，你也可以指定部分列进⾏重复项判断。假设我们还有⼀列值，且只希望根据`k1`列过滤重复项：

In [33]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated`和`drop_duplicates`默认保留的是第⼀个出现的值组合。传⼊`keep='last'`则保留最后⼀个：

In [34]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## 利用函数或映射进行数据转换
对于许多数据集，你可能希望根据数组、`Series`或`DataFrame`列中的值来实现转换⼯作。我们来看看下⾯这组有关⾁类的数据：

In [35]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


假设你想要添加⼀列表示该⾁类⻝物来源的动物类型。我们先编写⼀个不同⾁类到动物的映射：

In [36]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

`Series`的`map`⽅法可以接受⼀个函数或含有映射关系的字典型对象，但是这⾥有⼀个⼩问题，即有些⾁类的⾸字⺟⼤写了，⽽另⼀些则没有。因此，我们还需要使⽤`Series`的`str.lower`⽅法，将各个值转换为⼩写：

In [37]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [38]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


也可以传⼊⼀个能够完成全部这些⼯作的函数：

In [39]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

使⽤`map`是⼀种实现元素级转换以及其他数据清理⼯作的便捷⽅式。

## 替换值
利⽤`fillna`⽅法填充缺失数据可以看做值替换的⼀种特殊情况。前⾯已经看到，`map`可⽤于修改对象的数据⼦集，⽽`replace`则提供了⼀种实现该功能的更简单、更灵活的⽅式。我们来看看下⾯这个`Series`：

In [40]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

-999这个值可能是⼀个表示缺失数据的标记值。要将其替换为`pandas`能够理解的`NA`值，我们可以利⽤`replace`来产⽣⼀个新的`Series`（除⾮传⼊`inplace=True`）：

In [41]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

如果你希望⼀次性替换多个值，可以传⼊⼀个由待替换值组成的列表以及⼀个替换值：：

In [42]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

要让每个值有不同的替换值，可以传递⼀个替换列表：

In [43]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

传⼊的参数也可以是字典：

In [44]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## 重复名轴索引
跟`Series`中的值⼀样，轴标签也可以通过函数或映射进⾏转换，从⽽得到⼀个新的不同标签的对象。轴还可以被就地修改，⽽⽆需新建⼀个数据结构。接下来看看下⾯这个简单的例⼦：

In [45]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

跟`Series`⼀样，轴索引也有⼀个`map`⽅法：

In [46]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

你可以将其赋值给`index`，这样就可以对`DataFrame`进⾏就地修改：

In [47]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


如果想要创建数据集的转换版（⽽不是修改原始数据），⽐较实⽤的⽅法是`rename`：

In [48]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


`rename`可以结合字典型对象实现对部分轴标签的更新：

In [49]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


`rename`可以实现复制`DataFrame`并对其索引和列标签进⾏赋值。如果希望就地修改某个数据集，传⼊`inplace=True`即可：

In [50]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## 离散化和面元划分
为了便于分析，连续数据常常被离散化或拆分为“⾯元”（bin）。
假设有⼀组⼈员数据，⽽你希望将它们划分为不同的年龄组：

In [51]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

接下来将这些数据划分为“18到25”、“26到35”、“35到60”以及“60
以上”⼏个⾯元。要实现该功能，你需要使⽤`pandas`的`cut`函数：

In [52]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

`pandas`返回的是⼀个特殊的`Categorical`对象。结果展示了
`pandas.cut`划分的⾯元。你可以将其看做⼀组表示⾯元名称的字
符串。它的底层含有⼀个表示不同分类名称的类型数组，以及⼀
个`codes`属性中的年龄数据的标签：

In [53]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [54]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [55]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

`pd.value_counts(cats)`是`pandas.cut`结果的⾯元计数。  
跟“区间”的数学符号⼀样，圆括号表示开端，⽽⽅括号则表示闭
端（包括）。哪边是闭端可以通过`right=False`进⾏修改：

In [56]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

你可以通过传递⼀个列表或数组到`labels`，设置⾃⼰的⾯元名称：

In [57]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

如果向`cut`传⼊的是⾯元的数量⽽不是确切的⾯元边界，则它会
根据数据的最⼩值和最⼤值计算等⻓⾯元。下⾯这个例⼦中，我
们将⼀些均匀分布的数据分成四组

In [58]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.34, 0.55], (0.34, 0.55], (0.76, 0.97], (0.76, 0.97], (0.34, 0.55], ..., (0.34, 0.55], (0.34, 0.55], (0.55, 0.76], (0.34, 0.55], (0.12, 0.34]]
Length: 20
Categories (4, interval[float64]): [(0.12, 0.34] < (0.34, 0.55] < (0.55, 0.76] < (0.76, 0.97]]

选项`precision=2`，限定⼩数只有两位。  
`qcut`是⼀个⾮常类似于`cut`的函数，它可以根据样本分位数对数据
进⾏⾯元划分。根据数据的分布情况，`cut`可能⽆法使各个⾯元
中含有相同数量的数据点。⽽`qcut`由于使⽤的是样本分位数，因
此可以得到⼤⼩基本相等的⾯元：

In [59]:
data = np.random.randn(1000)  # 正态分布
cats = pd.qcut(data, 4)  # 分为四分数值
cats

[(-0.0265, 0.62], (0.62, 3.928], (-0.68, -0.0265], (0.62, 3.928], (-0.0265, 0.62], ..., (-0.68, -0.0265], (-0.68, -0.0265], (-2.9499999999999997, -0.68], (0.62, 3.928], (-0.68, -0.0265]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.68] < (-0.68, -0.0265] < (-0.0265, 0.62] < (0.62, 3.928]]

In [60]:
pd.value_counts(cats)

(0.62, 3.928]                   250
(-0.0265, 0.62]                 250
(-0.68, -0.0265]                250
(-2.9499999999999997, -0.68]    250
dtype: int64

与`cut`类似，你也可以传递⾃定义的分位数（0到1之间的数值，
包含端点）：

In [61]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.0265, 1.286], (-0.0265, 1.286], (-1.187, -0.0265], (-0.0265, 1.286], (-0.0265, 1.286], ..., (-1.187, -0.0265], (-1.187, -0.0265], (-2.9499999999999997, -1.187], (-0.0265, 1.286], (-1.187, -0.0265]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -1.187] < (-1.187, -0.0265] < (-0.0265, 1.286] < (1.286, 3.928]]

## 检测和过滤异常值
过滤或变换异常值（outlier）在很⼤程度上就是运⽤数组运算。  
来看⼀个含有正态分布数据的`DataFrame`：

In [62]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()  # describe()函数就是返回这两个核心数据结构的统计变量(Series和DataFrame)

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.049091,0.026112,-0.002544,-0.051827
std,0.996947,1.007458,0.995232,0.998311
min,-3.645860,-3.184377,-3.745356,-3.428254
25%,-0.599807,-0.612162,-0.687373,-0.747478
50%,0.047101,-0.013609,-0.022158,-0.088274
75%,0.756646,0.695298,0.699046,0.623331
max,2.653656,3.525865,2.735527,3.366626


假设你想要找出某列中绝对值⼤⼩超过3的值：

In [63]:
col = data[2]
col[np.abs(col) > 3]

41    -3.399312
136   -3.745356
Name: 2, dtype: float64

要选出全部含有“超过3或－3的值”的⾏，你可以在布尔型
`DataFrame`中使⽤`any`⽅法：

In [66]:
data[(np.abs(data) > 3).any(1)]  # any函数可以来判断真假，如果元素内只要有一个元素是真的，也就是非0，那么就返回true,否则返回false

,0,1,2,3
41,0.457246,-0.025907,-3.399312,-0.974657
60,1.951312,3.260383,0.963301,1.201206
136,0.508391,-0.196713,-3.745356,-1.520113
235,-0.242459,-3.056990,1.918403,-0.578828
258,0.682841,0.326045,0.425384,-3.428254
322,1.179227,-3.184377,1.369891,-1.074833
544,-3.548824,1.553205,-2.186301,1.277104
635,-0.578093,0.193299,1.397822,3.366626
782,-0.207434,3.525865,0.283070,0.544635
803,-3.645860,0.255475,-0.549574,-1.907459


根据这些条件，就可以对值进⾏设置。下⾯的代码可以将值限制
在区间－3到3以内：

In [67]:
data[np.abs(data) > 3] = np.sign(data) * 3  # sign()是Python的Numpy中的取数字符号（数字前的正负号）的函数
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.050286,0.025567,-0.001399,-0.051765
std,0.992920,1.004214,0.991414,0.995761
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.599807,-0.612162,-0.687373,-0.747478
50%,0.047101,-0.013609,-0.022158,-0.088274
75%,0.756646,0.695298,0.699046,0.623331
max,2.653656,3.000000,2.735527,3.000000


根据数据的值是正还是负，`np.sign(data)`可以⽣成1和-1：

In [70]:
np.sign(data).head()  # head()函数默认返回前五行数据

,0,1,2,3
0,-1.0,1.0,-1.0,1.0
1,1.0,-1.0,1.0,-1.0
2,1.0,1.0,1.0,-1.0
3,-1.0,-1.0,1.0,-1.0
4,-1.0,1.0,-1.0,-1.0


## 排列和随机采样
利⽤`numpy.random.permutation`函数可以轻松实现对`Series`或
`DataFrame`的列的排列⼯作（permuting，随机重排序）。通过
需要排列的轴的⻓度调⽤`permutation`，可产⽣⼀个表示新顺序
的整数数组：

In [75]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([3, 4, 2, 1, 0])

然后就可以在基于`iloc`的索引操作或`take`函数中使⽤该数组了：

In [76]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [77]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
4,16,17,18,19
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3


如果不想⽤替换的⽅式选取随机⼦集，可以在`Series`和
`DataFrame`上使⽤`sample`⽅法

In [81]:
df.sample(n=3)  # 选取随机子集

,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


要通过替换的⽅式产⽣样本（允许重复选择），可以传递
`replace=True`到`sample`：

In [82]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

4    4
4    4
3    6
4    4
1    7
0    5
3    6
4    4
3    6
1    7
dtype: int64

In [83]:
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

## 计算指标/哑变量
另⼀种常⽤于统计建模或机器学习的转换⽅式是：将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”。  
如果`DataFrame`的某⼀列中含有`k`个不同的值，则可以派⽣出⼀个`k`列矩阵或`DataFrame`（其值全为1和0）。`pandas`有⼀个`get_dummies`函数可以实现该功能（其实⾃⼰动⼿做⼀个也不难）。  
使⽤之前的⼀个`DataFrame`例⼦：

In [84]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [85]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


有时候，你可能想给指标`DataFrame`的列加上⼀个前缀，以便能
够跟其他数据进⾏合并。`get_dummies`的`prefix`参数可以实现该
功能：

In [86]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


如果`DataFrame`中的某⾏同属于多个分类，则事情就会有点复
杂。看⼀下`MovieLens 1M`数据集，14章会更深⼊地研究它：

In [88]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('./movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


要为每个`genre`添加指标变量就需要做⼀些数据规整操作。⾸
先，我们从数据集中抽取出不同的`genre`值：

In [90]:
all_genres = []
for x in movies.genres:
                                     # split() 通过指定分隔符对字符串进行切片，如果参数 num 有指定值，则分隔 num+1 个子字符串
    all_genres.extend(x.split('|'))  # extend() 函数用于在列表末尾一次性追加另一个序列中的多个值（用新列表扩展原来的列表）
genres = pd.unique(all_genres)       # unique()是以 数组形式（numpy.ndarray）返回列的所有唯一值（特征的所有唯一值）

现在有：

In [91]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

构建指标`DataFrame`的⽅法之⼀是从⼀个全零`DataFrame`开始：

In [94]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


现在，迭代每⼀部电影，并将`dummies`各⾏的条⽬设为1。要这
么做，我们使⽤`dummies.columns`来计算每个类型的列索引：

In [95]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [96]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

然后，根据索引，使⽤`.iloc`设定值：

In [97]:
# enumerate()函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

然后，和以前⼀样，再将其与`movies`合并起来：

In [99]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))  # 对于series，是给索引列加前缀，对于Dataframe，是给列名加前缀
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                               1
Genre_Children's                              1
                               ...             
Genre_War                                     0
Genre_Musical                                 0
Genre_Mystery                                 0
Genre_Film-Noir                               0
Genre_Western                                 0
Name: 0, Length: 21, dtype: object

注：对于很⼤的数据，⽤这种⽅式构建多成员指标变量就会变得⾮常慢。最好使⽤更低级的函数，将其写⼊`NumPy`数组，然后结果包装在`DataFrame`中。

⼀个对统计应⽤有⽤的秘诀是：结合`get_dummies`和诸如`cut`之
类的离散化函数：

In [101]:
np.random.seed(12345)  # seed()方法改变随机数生成器的种子，可以在调用其他随机模块函数之前调用此函数
values = np.random.rand(10)  # 本函数可以返回一个或一组服从“0~1”,"均匀分布"的随机样本值。随机样本取值范围是[0,1)，不包括1
values

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

In [103]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 字符串对象方法
对于许多字符串处理和脚本应⽤，内置的字符串⽅法已经能够满⾜要求了。例如，以逗号分隔的字符串可以⽤`split`拆分成数段：

In [104]:
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

`split`常常与`strip`⼀起使⽤，以去除空⽩符（包括换⾏符）：

In [106]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

利⽤加法，可以将这些⼦字符串以双冒号分隔符的形式连接起来

In [107]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

但这种⽅式并不是很实⽤。⼀种更快更符合`Python`⻛格的⽅式
是，向字符串"::"的`join`⽅法传⼊⼀个列表或元组

In [109]:
'::'.join(pieces)  # join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串。

'a::b::guido'

其它⽅法关注的是⼦串定位。检测⼦串的最佳⽅式是利⽤`Python`
的`in`关键字，还可以使⽤`index`和`find`：

In [110]:
'guido' in val

True

In [111]:
val.index(',')

1

In [116]:
val.find(':')

-1

注意`find`和`index`的区别：如果找不到字符串，`index`将会引发⼀
个异常（⽽不是返回－1）：

In [117]:
val.index(':')

ValueError: substring not found

与此相关，`count`可以返回指定⼦串的出现次数：

In [118]:
val.count(',')

2

`replace`⽤于将指定模式替换为另⼀个模式。通过传⼊空字符
串，它也常常⽤于删除模式：

In [119]:
val.replace(',', '::')

'a::b:: guido'

In [120]:
val.replace(',', '')

'ab guido'

![](./python内置的字符串方法.png)

`casefold` 将字符转换为⼩写，并将任何特定区域的变量字符组合
转换成⼀个通⽤的可⽐较形式。

## 正则表达式
正则表达式提供了⼀种灵活的在⽂本中搜索或匹配（通常⽐前者复杂）字符串模式的⽅式。正则表达式，常称作`regex`，是根据正则表达式语⾔编写的字符串。`Python`内置的`re`模块负责对字符串应⽤正则表达式。我将通过⼀些例⼦说明其使⽤⽅法。  
`re`模块的函数可以分为三个⼤类：模式匹配、替换以及拆分。当然，它们之间是相辅相成的。⼀个`regex`描述了需要在⽂本中定位的⼀个模式，它可以⽤于许多⽬的。我们先来看⼀个简单的例⼦：假设我想要拆分⼀个字符串，分隔符为数量不定的⼀组空⽩符（制表符、空格、换⾏符等）。描述⼀个或多个空⽩符的`regex`是`\s+`：

In [121]:
import re
text = "foo  bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

调⽤`re.split('\s+',text)`时，正则表达式会先被编译，然后再在`text`上调⽤其`split`⽅法。你可以⽤`re.compile`⾃⼰编译`regex`以得到⼀个可重⽤的`regex`对象:

In [122]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

如果只希望得到匹配`regex`的所有模式，则可以使⽤`findall`⽅法：

In [123]:
regex.findall(text)

['  ', '\t ', '  \t']

注：如果想避免正则表达式中不需要的转义（\），则可以
使⽤原始字符串字⾯量如r'C:\x'（也可以编写其等价
式'C:\x'）

如果打算对许多字符串应⽤同⼀条正则表达式，强烈建议通过`re.compile`创建`regex`对象。这样将可以节省⼤量的`CPU`时间。`match`和`search`跟`findall`功能类似。`findall`返回的是字符串中所有的匹配项，⽽`search`则只返回第⼀个匹配项。`match`更加严格，它只匹配字符串的⾸部。来看⼀个⼩例⼦，假设我们有⼀段⽂本以及⼀条能够识别⼤部分电⼦邮件地址的正则表达式：

In [124]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive(忽略案例使正则表达式不区分大小写)
regex = re.compile(pattern, flags=re.IGNORECASE)

对`text`使⽤`findall`将得到⼀组电⼦邮件地址：

In [125]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

`search`返回的是⽂本中第⼀个电⼦邮件地址（以特殊的匹配项对象形式返回）。对于上⾯那个`regex`，匹配项对象只能告诉我们模式在原字符串中的起始和结束位置：

In [127]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

`regex.match`则将返回`None`，因为它只匹配出现在字符串开头的模式：

In [128]:
print(regex.match(text))

None


相关的，`sub`⽅法可以将匹配到的模式替换为指定字符串，并返
回所得到的新字符串：

In [129]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



假设你不仅想要找出电⼦邮件地址，还想将各个地址分成3个部
分：⽤户名、域名以及域后缀。要实现此功能，只需将待分段的
模式的各部分⽤圆括号包起来即可：

In [130]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

由这种修改过的正则表达式所产⽣的匹配项对象，可以通过其
`groups`⽅法返回⼀个由模式各段组成的元组：

In [131]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

对于带有分组功能的模式，`findall`会返回⼀个元组列表：

In [132]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

`sub`还能通过诸如`\1`、`\2`之类的特殊符号访问各匹配项中的分组。符号`\1`对应第⼀个匹配的组，`\2`对应第⼆个匹配的组，以此类推：

In [133]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



![](./正则表达式方法.jpg)

## pandas的矢量化字符串函数
清理待分析的散乱数据时，常常需要做⼀些字符串规整化⼯作。更为复杂的情况是，含有字符串的列有时还含有缺失数据：

In [134]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [135]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

通过`data.map`，所有字符串和正则表达式⽅法都能被应⽤于（传⼊`lambda`表达式或其他函数）各个值，但是如果存在`NA`（null）就会报错。为了解决这个问题，`Series`有⼀些能够跳过`NA`值的⾯向数组⽅法，进⾏字符串操作。通过`Series`的`str`属性即可访问这些⽅法。例如，我们可以通过`str.contains`检查各个电⼦邮件地址是否含有`"gmail"`：

In [136]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

也可以使⽤正则表达式，还可以加上任意`re`选项（如
`IGNORECASE`）：

In [137]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [144]:
x = data.str.findall(pattern, flags=re.IGNORECASE)
x

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

有两个办法可以实现⽮量化的元素获取操作：要么使⽤`str.get`，要么在`str`属性上使⽤索引：

In [145]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

要访问嵌⼊列表中的元素，我们可以传递索引到这两个函数中：

In [146]:
x.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [147]:
x.str[0]

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

你可以利⽤这种⽅法对字符串进⾏截取：

In [148]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

部分矢量化字符串方法：  
![](./部分⽮量化字符串⽅法.jpg)